# Web Scraping de Cotações das Ações Brasileiras  

### Este código realiza **web scraping** para obter os valores de cotação das **cinco principais ações brasileiras** listadas no site **Investing**.  

### 📌 Fluxo do Código  

1. **Automação do Navegador** → Utiliza a biblioteca **Selenium** para acessar o site e navegar até as páginas das ações.  
2. **Extração de Dados** → Captura tabelas de cotações usando **BeautifulSoup**.  
3. **Tratamento dos Dados** → Usa **Pandas** para limpar e formatar os dados, convertendo datas e valores numéricos.  
4. **Armazenamento** → Salva os dados em um arquivo CSV.  
5. **Dashboard** → Os dados serão exibidos posteriormente em um dashboard usando **Streamlit**.  

⚠ **Observação:** O código foi executado com sucesso em **12/02/2025**. Caso o **DOM do Investing** seja alterado, pode ser necessário atualizar os seletores para garantir o funcionamento.  


In [1]:
# Importação das bibliotecas necessárias

import selenium  # Biblioteca para automação de navegadores.
import pandas as pd  # Biblioteca para manipulação de dados em DataFrames (Tabelas).

# Importação dos módulos do Selenium
from selenium import webdriver  # Permite controlar um navegador via código.
from selenium.webdriver.chrome.service import Service  # Garante a inicialização do ChromeDriver.
from selenium.webdriver.common.by import By  # Fornece métodos para localizar elementos na página.
from selenium.webdriver.support.ui import WebDriverWait  # Permite aguardar a presença de elementos na página para prosseguir com a execução do código.
from selenium.webdriver.support import expected_conditions as EC  # Define condições de espera para elementos.

# Biblioteca BeautifulSoup para análise e extração de dados de HTML.
from bs4 import BeautifulSoup  

# Módulo time para pausas no código (evita bloqueios por solicitações excessivas).
import time  

# Biblioteca datetime para manipulação de datas e horários.
from datetime import datetime  

# Biblioteca locale para configuração regional (idioma, formatação de números e datas).
import locale  

# Biblioteca os para interações com o sistema operacional (como manipulação de arquivos e diretórios).
import os  

# Define o locale para português do Brasil.
locale.setlocale(locale.LC_TIME, "pt_BR.UTF-8")

'pt_BR.UTF-8'

In [2]:
# Cria um objeto Service. Ele é necessário para gerenciar o processo do driver do Chrome.
service = Service()

# Cria um objeto ChromeOptions, que permite configurar várias opções do navegador Chrome.
options = webdriver.ChromeOptions()

# Inicializa o WebDriver do Chrome com as opções configuradas e o serviço.
# Isso abre o navegador Chrome controlado pelo Selenium.
driver = webdriver.Chrome(service=service, options=options)

# Define a URL que você deseja acessar com o navegador, nesse exemplo, o Investing.
url = 'https://br.investing.com/indices/brazil-indices'

# Faz com que o driver abra a página da URL especificada.
driver.get(url)

# Maximiza a janela do navegador para tela cheia.
driver.maximize_window()

# Tenta encontrar o elemento de fechar a pop-up de login, caso não encontre (o elemento não apareceu), proseegue com a execução
try:
    # Fecha a pop-up de cadastro que aparece ao acessar o Investing.
    closePopUp = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'float-end')))
    closePopUp.click()
    
except:
    pass

In [3]:
# Define uma variável tableList como uma lista vazia. Essa lista vai armazenar o HTML da tabela capturada.
tableList = []

# Capta todos os elementos de tag <a> no intervalo entre 444 e 449 no HTML da página e os armazena na lista elements.
# Isso seleciona uma faixa específica de links na página.
elements = driver.find_elements(By.TAG_NAME, 'a')[444:449]

# Define uma variável elementName como uma lista vazia. Essa lista vai armazenar os nomes dos elementos capturados.
elementName = []

# Cria um laço de repetição para iterar sobre os elementos da lista elements.
for i, element in enumerate(elements):

    try:

        # Clica no elemento atual da lista elements.
        elements[i].click()

        # Espera até que o elemento de "historic" esteja presente e clica nele. Este é um link de navegação.
        historic = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((
                By.XPATH, f'//*[@id="__next"]/div[2]/div[2]/div[2]/div[1]/nav/div[2]/ul/li[3]/a')))

        historic.click()

        # Espera até que o elemento de "date" (data) esteja visível e clica nele.
        date = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((
                By.XPATH, '//*[@id="__next"]/div[2]/div[2]/div[2]/div[1]/div[2]/div[2]/div[2]/div[2]')))
        
        date.click()

        # Captura o campo de entrada (input) e interage com ele.
        inputElement = driver.find_elements(By.TAG_NAME, 'input')[1]  # Seleciona o segundo campo <input>
        inputElement.click()  # Clica no campo de input
        inputElement.clear()  # Limpa o valor do campo de input
        inputElement.send_keys('020012020')  # Preenche o campo de input com a data específica (01/01/2020)

        # Espera até que o botão de "apply" esteja presente e clica nele.
        apply = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((
                By.XPATH, f'//*[@id="__next"]/div[2]/div[2]/div[2]/div[1]/div[2]/div[2]/div[2]/div[3]/div[2]')))
        
        apply.click()

        # Aguarda 2 segundos para garantir que a página seja atualizada.
        time.sleep(2)
        
        # Captura a tabela na página após a ação e a armazena em tableList.
        table = driver.find_element(By.XPATH, '//*[@id="__next"]/div[2]/div[2]/div[2]/div[1]/div[2]/div[3]/table')
        tableList.append(table.get_attribute('outerHTML'))  # Adiciona o HTML da tabela à lista tableList

        # Aguarda 2 segundos antes de voltar para a página anterior.
        time.sleep(2)

        # Volta para a página anterior (duas vezes).
        driver.back()
        driver.back()

        # Aguarda 2 segundos após as navegações.
        time.sleep(2)
        
        # Recarrega os elementos da lista elements, para garantir que a seleção está atualizada.
        elements = driver.find_elements(By.TAG_NAME, 'a')[444:449]
        
        # Imprime uma mensagem informando que os dados foram capturados com sucesso.
        print(f'Dados da {elements[i].text} captados com sucesso!')
        
        # Adiciona o nome do elemento (text) à lista elementName.
        elementName.append(elements[i].text)
        
        # Aguarda 2 segundos antes de continuar para o próximo loop.
        time.sleep(2)
    
    except Exception as e:
        # Em caso de erro, imprime uma mensagem de erro.
        print(f'Erro na captação de dados do item {i} da tabela!')


Dados da Ibovespa captados com sucesso!
Dados da IBrX 50 captados com sucesso!
Dados da IBrX 100 captados com sucesso!
Dados da Brasil Amplo IBrA captados com sucesso!
Dados da MidLarge Cap MLCX captados com sucesso!


In [ ]:
# Define um cabeçalho para as colunas do DataFrame que será gerado. Esse cabeçalho corresponde aos dados que esperamos extrair da tabela.
headers = ['data', 'ultimo', 'abertura', 'maxima', 'minima', 'volume', 'variacao', 'acao']
# Cria uma lista vazia para armazenar as linhas de dados extraídas da tabela.
tableDb = []
# Inicia um loop para iterar sobre cada tabela armazenada na lista 'tableList'.
for index, tableData in enumerate(tableList):

    # Utiliza BeautifulSoup para analisar o HTML de cada tabela, permitindo a extração dos dados.
    soup = BeautifulSoup(tableData, 'html.parser')
    
    # Encontra todas as linhas da tabela (tr) no HTML da tabela.
    tableRows = soup.find_all('tr')
    

    # Itera sobre cada linha da tabela.
    for i, row in enumerate(tableRows):
        rowData = []  # Cria uma lista para armazenar os dados da linha atual.
        
        # Encontra todas as células da linha (th e td), que contêm os dados.
        cells = row.find_all(['th', 'td'])
        
        # Itera sobre cada célula e adiciona o conteúdo (texto) da célula à lista de dados da linha.
        for c, cell in enumerate(cells):
            rowData.append(cell.get_text(strip=True))  # 'strip=True' remove espaços em branco extras.
        
        if i == 0:
            # Se for a primeira iteração da tabela correspondente ação, a primeira linha será gravada com o título do cabeçalho
            rowData.append('acao')
        else:
            # Se não adiciona o nome do elemento (nome da ação vindo da lista elementName) ao final dos dados da linha.
            rowData.append(elementName[index])
        
        # Adiciona a linha com os dados extraídos à lista 'tableDb'.
        tableDb.append(rowData)

# Verifica se a tabela contém dados antes de prosseguir.
if tableDb:

    # Converte a lista 'tableDb' em um DataFrame do pandas, usando o cabeçalho definido anteriormente.
    df = pd.DataFrame(tableDb, columns=headers).drop_duplicates()  # Remove as linhas duplicadas, no caso, as que contém o cabeçalho da tabela HTML.
    df = df[1:] # Remove a primeira linha
    # Converte a coluna 'data' para o formato de data, substituindo pontos por barras.
    df['data'] = df['data'].str.replace('.', '/')
    df['data'] = pd.to_datetime(df['data'], format='%d/%m/%Y')

    # Extração do mês e o ano da coluna 'data'.
    df['mes'] = df['data'].dt.strftime('%b')
    df['ano'] = df['data'].dt.strftime('%Y')

    # Remove o símbolo de porcentagem da coluna 'variacao' e converte os valores para um formato numérico.
    df['variacao'] = df['variacao'].str.replace('%', '')
    
    # Para as colunas numéricas ('ultimo', 'abertura', 'maxima', 'minima', 'variacao'), realiza o seguinte:
    for column in ['ultimo', 'abertura', 'maxima', 'minima', 'variacao']:
        df[column] = df[column].str.replace('.', '', regex=False)  # Remove os pontos (caso existam como separadores de milhar).
        df[column] = df[column].str.replace(',', '.', regex=False)  # Substitui vírgulas por pontos (caso existam como separadores decimais).
        df[column] = pd.to_numeric(df[column])  # Converte os valores para numérico.

    # Converte a coluna 'variacao' de percentual para um número decimal (dividindo por 100).
    df['variacao'] = df['variacao'] / 100

    # Remove a coluna 'volume', pois não é necessária para análise.
    df = df.drop(columns=['volume'])
# Salva os dados extraídos e processados em um arquivo CSV (sem os índices das linhas) chamado 'Dados Investing.csv', usando ponto e vírgula como separador.
df.to_csv('DadosInvesting.csv', sep=',', index=False)
